In [138]:
import pandas as pd
import numpy as np
import seaborn as sns  
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from scipy.stats import pearsonr

localId,TalkerId,Type,SubType,IsSender,CreateTime,Status,StrContent,StrTime,Remark,NickName,Sender

In [ ]:
# 从CSV文件中读取数据，并处理时间和昵称信息
mydf = pd.read_csv('OPClub_cut.csv')
Strtime = mydf.StrTime.tolist()
Createtime = mydf.CreateTime.tolist()
dic_time = dict(zip(Createtime,Strtime))
nicknamelst = mydf.NickName.tolist()
dict_name = dict(zip(Createtime,nicknamelst))
nickname= list(dict.fromkeys(nicknamelst))  
dic_nameint = dict(zip(nickname,range(len(nickname))))
dic_intname = dict(zip(range(len(nickname)),nickname))
print(nickname)

In [140]:
# # 移除'Occupation'列中值为'Engineer'的所有行
# filtered_df = mydf[mydf['NickName'] != None]

# # 保存修改后的数据
# filtered_df.to_csv('OPClub_cut.csv', index=False)

In [141]:
# 示例一维数据点  
data = Createtime

# 设定距离阈值δ   
delta = 300

# 初始化一个空列表，用于存储聚类结果  
clusters = []  
current_cluster = []  

# 遍历数据点  
for i in range(len(data)):  
    # 如果current_cluster为空，或者当前点与上一个聚类点的距离小于等于delta  
    if current_cluster  == [] or (len(current_cluster) > 0 and abs(data[i] - current_cluster[-1] <= delta)):  
        # 添加到当前聚类  
        current_cluster.append(data[i])  
    else:  
        # 否则，保存当前聚类并开始新的聚类  
        clusters.append(current_cluster)  
        current_cluster = [data[i]]  

# 保存最后一个聚类（如果有的话）  
if current_cluster:  
    clusters.append(current_cluster)

In [ ]:
print((clusters[1]))
# 存的是CreateTime

In [143]:
timemap = []
# 抛弃那些只有一个人说了话的话题
for i in clusters:
    if len(i)>=2:
        timemap.append(i)

In [ ]:
print((timemap[0]))
print((timemap[1]))
print((timemap[2]))
# 存的是CreateTime

In [145]:
# 实现每个人的话题参与向量
# 描述很多个人在很多个话题中的发言情况时，如果一个人对某个话题说了话则标记为1，没说话则标记为0

# 这个列表推导式用于创建一个二维列表 data，其中：
# 行数（外层列表的长度）是话题参与者的数量（即 nickname 的长度）。z
# 列数（内层列表的长度）是话题的数量（即 timemap 的长度）。i
# 初始值都设置为0，表示尚未参与。
data = [[0 for i in range(len(timemap))] for z in range(len(nickname))]
for k,i in enumerate(timemap):# k 是当前话题的索引（整数），i 是list[CreateTime]
    namelst =  [dict_name[q] for q in i]
    for p,z in enumerate(nickname):
        if z in namelst:
            data[p][k]=1

In [ ]:
sub_matrix = [row[0:10] for row in data[0:]]

# 打印提取的部分
for row in sub_matrix:
    print(row)

In [147]:
# data1，2都是n×n的矩阵，data1的第i行第j列意味着n个人里面第i个人的话题参与向量预测第j个人的话题参与向量得到的auc值

# data2的第i行第j列意味着n个人里面第第i个人参与了的话题，第j个人参与了多少（百分比占比多少）

data1=[[0 for i in range(len(nickname))] for j in range(len(nickname))]

data2=[[0 for i in range(len(nickname))] for j in range(len(nickname))]

for i,j in enumerate(nickname):

    for p,q in enumerate(nickname):

        data1[i][p] = roc_auc_score(data[i], data[p])

        count1 = 0

        count2 = 0

        for k in range(len(timemap)):

            if  data[i][k]==1:

                count2+=1

            if  data[i][k]==1 and data[p][k]==1:

                count1+=1

        # data3[i][p] = count1/count2


# 也可以用接话程度去描述一个人是不是喜欢另一个人，我把每个话题，谁接这个人话接最多
# 那统计出来的矩阵人A和人B对应的值就会+1，最后总体来看谁接这个人话最多

# data3 = [[0 for i in range(len(nickname))] for z in range(len(nickname))]

# for i in timemap:# 对于每个对话

#     namelst = [dict_name[p] for p in i]
#     # print(namelst)

#     for k in range(len(nickname)):# 对于每个人

#         q_lst = [0 for i in range(len(nickname))]# 记录每个人接了多少对话

#         for z,j in enumerate(namelst):

#             if z == len(i)-1:
#                 break

#             if j == dic_intname[k]:# 找到了对话中的第k个人
#                 q_lst[k]+=1

#         print(q_lst)

#         # q_lst[k]=0

#         for i1,i2 in enumerate(q_lst):

#             if i2 == max(q_lst) and max(q_lst) != 0:

#                 data3[k][i1]+=1

In [148]:
# print(data3[2][4])

In [ ]:
for i in range(len(nickname)):

    data1[i][i]=0

map = pd.DataFrame(data1)

plt.figure(figsize=(8, 8))  

plt.rcParams['font.sans-serif'] = ['SimHei']

realname = [i for i in nickname]

sns.heatmap(map, annot=True, cmap="YlGnBu")  

plt.title('OPClub')  

plt.xticks(ticks=range(len(nickname)), labels=nickname, rotation=90)  # 旋转90度以便竖着显示

plt.yticks(ticks=range(len(nickname)), labels=nickname, rotation=360) 

plt.xlabel('群友')  

plt.ylabel('群友')  

plt.show()